In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib.ticker import MultipleLocator

FOR_PRINT = True

if FOR_PRINT:
    LINE_WIDTH = 1
    MARKER_SIZE = 3
    FONT_SIZE = 8

    AXES_WIDTH = 0.65 * LINE_WIDTH

    plt.rcParams['grid.linewidth']=AXES_WIDTH
    plt.rcParams['axes.linewidth']=AXES_WIDTH
    plt.rcParams['axes.labelpad']=3.0

    plt.rcParams['xtick.major.pad']=0
    plt.rcParams['xtick.major.size']=2.0
    plt.rcParams['xtick.major.width']=AXES_WIDTH
    plt.rcParams['xtick.minor.size']=1.0
    plt.rcParams['xtick.minor.width']=0.75 * AXES_WIDTH

    plt.rcParams['ytick.major.pad']=-1.5
    plt.rcParams['ytick.major.size']=2.0
    plt.rcParams['ytick.major.width']=AXES_WIDTH
    plt.rcParams['ytick.minor.size']=1.0
    plt.rcParams['ytick.minor.width']=0.75 * AXES_WIDTH
else:
    LINE_WIDTH = 6
    MARKER_SIZE = 14
    FONT_SIZE = 45

%matplotlib inline
#plt.rcParams['figure.figsize'] = [15, 15]
plt.rcParams['lines.linewidth'] = LINE_WIDTH
plt.rcParams['lines.markeredgewidth'] = 0.75 * LINE_WIDTH
plt.rcParams['lines.markersize'] = MARKER_SIZE
plt.rcParams['font.size'] = FONT_SIZE
rc('text', usetex=True)

In [ ]:
data = list()
with open("condition_numbers.json") as results_json_file:
    data = json.load(results_json_file)

In [ ]:
#elements = ["Tet4", "Tet10", "Hex8", "Hex20"]
elements = ["Tet4", "Tet10", "Hex8"]
stabilization_factors = list(set([obj['stabilization_factor'] for obj in data]))
preconditioners = list(set([obj['preconditioner_name'] for obj in data]))
legend_loc = ['upper right', 'upper left']

In [ ]:
def get_epsilon_index(obj):
    return obj['epsilon_index']

def should_keep_preconditioner(element, preconditioner):
    if element == 'Hex8':
        return preconditioner == 'Diagonal'
        #return (element in ['Tet4', 'Hex8'] and preconditioner == 'Diagonal') \
        #or (element in ['Tet10', 'Hex20'] and preconditioner == 'Schwarz')
    else:
        return True
    
def get_element_name(element):
    if element == 'Tet4':
        return 'Linear Tet'
    elif element == 'Tet10':
        return 'Quadratic Tet'
    elif element == 'Hex8':
        return 'Trilinear Hex'
    return element

FIG_WIDTH = 3.6 if FOR_PRINT else 20
FIG_HEIGHT = 2.0 if FOR_PRINT else 10

filenames = {
    0.0: "condition_number_no_stabilization.pgf",
    1e-06: "condition_number_stabilized.pgf"
}

for (loc, stab_factor) in zip(legend_loc, stabilization_factors):
    fig = plt.figure(figsize=(FIG_WIDTH, FIG_HEIGHT))
    
    for element in elements:
        for preconditioner in preconditioners:
            if should_keep_preconditioner(element, preconditioner):
                sorted_objects = [obj for obj in data if obj['element_name'] == element
                                  and obj['stabilization_factor'] == stab_factor
                                  and obj['preconditioner_name'] == preconditioner]
                sorted_objects.sort(key=get_epsilon_index)
                eps_indices = [obj['epsilon_index'] for obj in sorted_objects]
                volume_fraction = [2**(-r) for r in eps_indices]
                mass_condition = [obj['mass_condition']['condition_number'] for obj in sorted_objects]
                stiffness_condition = [obj['stiffness_condition']['condition_number'] for obj in sorted_objects]
                system_condition = [obj['system_condition']['condition_number'] for obj in sorted_objects]

                preconditioner_name = preconditioner if preconditioner else "Unchanged"
                #label_suffix = " {} ({}) (stab = {:1.1e})".format(element, preconditioner_name, stab_factor)
                label_suffix = " {} ({})".format(get_element_name(element), preconditioner_name, stab_factor)
#                 plt.plot(volume_fraction, mass_condition, label=r'$\kappa(M)$' + label_suffix)
#                 plt.plot(volume_fraction, stiffness_condition, label='$\kappa(K)$' + label_suffix)
                plt.plot(volume_fraction, system_condition, marker='o', label=label_suffix)
                plt.legend(prop={'size': 0.75 * FONT_SIZE}, loc =loc)

    plt.grid()
    plt.xscale('log')
    plt.yscale('log')
    plt.xlabel('Volume fraction', fontsize=FONT_SIZE)
    #plt.ylabel('$\kappa(P(M + (\Delta t)^2 K))$', fontsize=45)
    plt.ylabel('$\kappa(SA)$', fontsize=FONT_SIZE, labelpad=plt.rcParams['axes.labelpad']/2)
    plt.tick_params(axis='both', which='major', labelsize=FONT_SIZE)
    if stab_factor == 0.0:
        #plt.title("Condition number, no stabilization", fontsize=45)
        pass
    else:
        #plt.title("Condition number, stabilization factor ${}$".format(stab_factor), fontsize=45)
        pass
    plt.tight_layout()
    #plt.savefig('condition_number_stab_{}.pdf'.format(stab_factor), format='pdf', bbox_inches='tight')
    filename = 'condition_number_stab_{}.pgf'.format(stab_factor)
    if stab_factor in filenames:
        filename = filenames[stab_factor]
    plt.savefig(filename, format='pgf', bbox_inches='tight')
    plt.show()